# Download data

In [1]:
from file_download import *
import tensorflow as tf

#train_inputs, train_labels
#test_inputs, test_labels are now loaded. 




Found and verified train-images-idx3-ubyte.gz
Found and verified train-labels-idx1-ubyte.gz
Found and verified t10k-images-idx3-ubyte.gz
Found and verified t10k-labels-idx1-ubyte.gz

Reading files train-images-idx3-ubyte.gz and train-labels-idx1-ubyte.gz
60000 28 28
(Images) Returned a tensor of shape  (60000, 784)
(Labels) Returned a tensor of shape: 60000
Sample labels:  [5 0 4 1 9 2 1 3 1 4]

Reading files t10k-images-idx3-ubyte.gz and t10k-labels-idx1-ubyte.gz
10000 28 28
(Images) Returned a tensor of shape  (10000, 784)
(Labels) Returned a tensor of shape: 10000
Sample labels:  [7 2 1 0 4 1 4 9 5 9]


/root/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Defining hyperparameters and constants

In [19]:
WEIGHTS_STRING = 'weights'
BIAS_STRING = 'bias'

batch_size = 100

img_width, img_height = 28,28
input_size = img_height * img_width
print('input_size::::',input_size)

num_labels = 10

# resets the default graph Otherwise raises an error about already initialized variables
tf.reset_default_graph()

input_size:::: 784


In [20]:
# Defining inputs and outputs
# Defining in placeholders as these items will come on the fly while running the session.
tf_inputs  = tf.placeholder(shape=[batch_size,input_size],dtype=tf.float32,name='inputs')
tf_labels = tf.placeholder(shape=[batch_size,num_labels],dtype=tf.float32,name='labels')


# Building layers with scoping

In [21]:
#By means of scoping, we name variables (weights and biases) for each layer as,layer1/weights , layer1/bias , layer2/weights , layer2/bias , output/weights ,and output/bias.



def define_net_parameters():
    '''
    1) A fully-connected layer with ReLU activation ( layer1 )
    2) A fully-connected layer with ReLU activation ( layer2 )
    3) A fully-connected softmax layer ( output )
    '''

    with tf.variable_scope('layer_1'):
        tf.get_variable(name=WEIGHTS_STRING,shape=[input_size,500],initializer=tf.random_normal_initializer(0,0.02))
        
        tf.get_variable(name=BIAS_STRING,shape=[500],initializer=tf.random_normal_initializer(0,0.1))
    
    with tf.variable_scope('layer_2'):
        tf.get_variable(name=WEIGHTS_STRING,shape=[500,250],initializer=tf.random_normal_initializer(0,0.2))
        
        tf.get_variable(name=BIAS_STRING,shape=[250],initializer=tf.random_normal_initializer(0,0.1))
        
    with tf.variable_scope('output'):
        
        tf.get_variable(name=WEIGHTS_STRING,shape=[250,10],initializer=tf.random_normal_initializer(0,0.2))
        
        tf.get_variable(name=BIAS_STRING,shape=[10],initializer=tf.random_normal_initializer(0,0.1))
        
        
        
        
    

# Inference

In [22]:
# Calling all the parameters
define_net_parameters()



def inference(x):
    
    with tf.variable_scope('layer_1',reuse = True):
        w,b = tf.get_variable(WEIGHTS_STRING),tf.get_variable(BIAS_STRING)
        tf_h1 = tf.nn.relu(tf.matmul(x,w)+b,name='hidden_1')
    
    
    with tf.variable_scope('layer_2',reuse = True):
        w,b = tf.get_variable(WEIGHTS_STRING),tf.get_variable(BIAS_STRING)
        tf_h2 = tf.nn.relu(tf.matmul(tf_h1,w)+b,name='hidden_2')
    
    
    
    with tf.variable_scope('output',reuse = True):
        w,b = tf.get_variable(WEIGHTS_STRING),tf.get_variable(BIAS_STRING)
        tf_h3 = tf.nn.bias_add(tf.matmul(tf_h2,w),b,name='hidden_3')
        
        
    return tf_h3
    

# Defining loss and optimizer

In [23]:

tf_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=inference(tf_inputs),labels=tf_labels))



In [24]:
tf_loss_minimize = tf.train.MomentumOptimizer(learning_rate=0.01,momentum=0.9,use_nesterov=True).minimize(tf_loss)

# Predictions

In [25]:
# defining predictions
tf_predictions = tf.nn.softmax(inference(tf_inputs))

# Running Neural Net

In [26]:
session = tf.InteractiveSession()
tf.global_variables_initializer().run()
NUM_EPOCH = 50

def accuracy(predictions, labels):
    return np.sum(np.argmax(predictions,axis=1).flatten()==labels.flatten())/batch_size


for epoch in range(NUM_EPOCH):
    train_loss = []
    
    for step in range(train_inputs.shape[0]//batch_size):
        # Creating one-hot encoded labels with labels
        # One-hot encoding dight 3 for 10-class MNIST data set will result in
        # [0,0,0,1,0,0,0,0,0,0]
        labels_one_hot = np.zeros((batch_size, num_labels),dtype=np.float32)
        labels_one_hot[np.arange(batch_size),train_labels[step*batch_size:(step+1)*batch_size]] = 1.0
        
        # Running the optimization process
        loss, _ = session.run([tf_loss,tf_loss_minimize],feed_dict={tf_inputs: train_inputs[step*batch_size: (step+1)*batch_size,:],tf_labels: labels_one_hot})
        train_loss.append(loss)
        
        
    test_accuracy = []
    # Testing Phase
    for step in range(test_inputs.shape[0]//batch_size):
        test_predictions = session.run(tf_predictions,feed_dict={tf_inputs: test_inputs[step*batch_size: (step+1)*batch_size,:]})
    
        batch_test_accuracy = accuracy(test_predictions,test_labels[step*batch_size: (step+1)*batch_size])
        test_accuracy.append(batch_test_accuracy)
    print('Average train loss for the %d epoch: %.3f\n'%(epoch+1,np.mean(train_loss)))
    print('\tAverage test accuracy for the %d epoch:%0.3f\n'%(epoch+1,np.mean(test_accuracy)*100.0))
    
session.close()
        
        
        
        
        
        
    

/root/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py:1645: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Average train loss for the 1 epoch: 0.307

	Average test accuracy for the 1 epoch:94.750

Average train loss for the 2 epoch: 0.088

	Average test accuracy for the 2 epoch:96.630

Average train loss for the 3 epoch: 0.052

	Average test accuracy for the 3 epoch:96.890

Average train loss for the 4 epoch: 0.032

	Average test accuracy for the 4 epoch:97.430

Average train loss for the 5 epoch: 0.020

	Average test accuracy for the 5 epoch:97.610

Average train loss for the 6 epoch: 0.013

	Average test accuracy for the 6 epoch:97.760

Average train loss for the 7 epoch: 0.009

	Average test accuracy for the 7 epoch:97.880

Average train loss for the 8 epoch: 0.006

	Average test accuracy for the 8 epoch:97.940

Average train loss for the 9 epoch: 0.004

	Average test accuracy for the 9 epoch:97.980

Average train loss for the 10 epoch: 0.003

	Average test accuracy for the 10 epoch:98.020

Average train loss for the 11 epoch: 0.002

	Average test accuracy for the 11 epoch:98.050

Averag